In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

In [ ]:
Images_Doc_Path = Path("../input/best-artworks-of-all-time/images/images")

In [ ]:
Image_Path = list(Images_Doc_Path.glob(r"*/*.jpg"))

In [ ]:
Image_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Image_Path))

In [ ]:
Image_Path_Series = pd.Series(Image_Path,name="JPG").astype(str)

In [ ]:
Image_Labels_Series = pd.Series(Image_Labels,name="CATEGORY")

In [ ]:
Main_Train_Data = pd.concat([Image_Path_Series,Image_Labels_Series],axis=1)

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
Train_Data,Test_Data = train_test_split(Main_Train_Data,train_size=0.8,shuffle=True,random_state=42)

In [ ]:
print(Train_Data.shape)

In [ ]:
print(Train_Data.head(-1))

In [ ]:
discriminator = Sequential()
discriminator.add(Flatten(input_shape=[200,200]))
discriminator.add(Dense(100,activation="relu"))
discriminator.add(Dense(50,activation="relu"))
discriminator.add(Dense(1,activation="sigmoid"))

In [ ]:
discriminator.compile(loss="binary_crossentropy",optimizer="adam")

In [ ]:
discriminator.trainable = False

In [ ]:
generator = Sequential()
generator.add(Dense(50,activation="relu",input_shape=[200]))
generator.add(Dense(100,activation="relu"))
generator.add(Dense(150,activation="relu"))
generator.add(Dense(550,activation="relu"))
generator.add(Dense(40000,activation="relu"))
generator.add(Reshape([200,200]))

In [ ]:
GAN = Sequential([generator,discriminator])

In [ ]:
GAN.compile(loss="binary_crossentropy",optimizer="adam")

In [ ]:
new_IMG_set = []
for imm in Train_Data["JPG"]:
    x_img = cv2.imread(imm,0)
    x_img = cv2.resize(x_img,(200,200))
    new_IMG_set.append(x_img)

In [ ]:
plt.imshow(new_IMG_set[0])

In [ ]:
print(new_IMG_set[0].shape)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(new_IMG_set).shuffle(buffer_size=1000)

In [ ]:
print(type(dataset))

In [ ]:
batch_size=1

In [ ]:
dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)

In [ ]:
print(dataset)

In [ ]:
epochs=5

In [ ]:
print(GAN.layers)
print("---"*10)
print(GAN.layers[1].layers)
print("---"*10)
print(GAN.layers[0].layers)

In [ ]:
print(GAN.layers[1].summary())
print("---"*10)
print(GAN.layers[0].summary())

In [ ]:
generator,discriminator = GAN.layers

for epoch in range(epochs):
    print(f"Current on {epoch+1}")
    i = 0
    for x_batch in dataset:
        i = i+1
        if i % 100 == 0:
            print(f"Currently on batch {i} of {len(new_IMG_set)//batch_size}")


            noise =  tf.random.normal(shape=[batch_size,200])
            gen_images = generator(noise)

            x_fake_or_real = tf.concat([gen_images,tf.dtypes.cast(x_batch,tf.float32)],axis=0)

            y1 = tf.constant([[0.0]]*batch_size + [[1.0]]*batch_size)

            discriminator.trainable = True
            discriminator.train_on_batch(x_fake_or_real,y1)



            noise = tf.random.normal(shape=[batch_size,200])
            y2 = tf.constant([[1.0]]*batch_size)
            discriminator.trainable = False
            GAN.train_on_batch(noise,y2)


In [ ]:
noise_Two = tf.random.normal(shape=[51,200])

In [ ]:
print(noise_Two.shape)

In [ ]:
plt.imshow(noise_Two)

In [ ]:
IMG_Gen = generator(noise_Two)

In [ ]:
print(IMG_Gen.shape)

In [ ]:
plt.imshow(IMG_Gen[:,40])

In [ ]:
plt.imshow(IMG_Gen[4,:20])

In [ ]:
plt.imshow(IMG_Gen[:,10])

In [ ]:
plt.imshow(IMG_Gen[1])

In [ ]:
figure = plt.figure(figsize=(10,10))
plt.imshow(IMG_Gen[9])